<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0" width=150px> </div>
    <div style="float: left; margin-left: 10px;"> <h1>LLMs for Data Science</h1>
<h1>Generative AI</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint
from datetime import datetime
import json

import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt 
import sqlite3

import openai
from openai import OpenAI

import transformers
from transformers import pipeline
from transformers import set_seed
set_seed(42) # Set the seed to get reproducible results

import langchain
import langchain_openai
from langchain_openai import ChatOpenAI
import langchain_core
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

import os
import gzip

import tqdm as tq
from tqdm.notebook import tqdm

import watermark

%load_ext watermark
%matplotlib inline

We start by printing out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.11.7
IPython version      : 8.12.3

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 24.3.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: 03802c3bf87993c3670c2fd8bf86e59d3d60bdfd

watermark       : 2.4.3
langchain       : 0.2.2
transformers    : 4.41.1
matplotlib      : 3.8.0
openai          : 1.30.5
sqlite3         : 2.6.0
langchain_core  : 0.2.3
pandas          : 2.2.3
numpy           : 1.26.4
langchain_openai: 0.1.8
tqdm            : 4.66.4
json            : 2.0.9



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

# OpenAI Basic Usage

The first step is generate API key on the OpenAI website and store it as the "OPENAI_API_KEY" variable in your local environment. Without it we won't be able to do anything. You can find your API key in your using settings: https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key

Then we are ready to instantiate the client

In [4]:
client = OpenAI()

We start by getting a list of supported models.

In [5]:
client.models.list()

SyncPage[Model](data=[Model(id='gpt-4o-audio-preview-2024-12-17', created=1734034239, object='model', owned_by='system'), Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'), Model(id='gpt-4o-realtime-preview-2024-12-17', created=1733945430, object='model', owned_by='system'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='gpt-4o-audio-preview-2024-10-01', created=1727389042, object='model', owned_by='system'), Model(id='gpt-4o-transcribe', created=1742068463, object='model', owned_by='system'), Model(id='gpt-4o-realtime-preview-2024-10-01', created=1727131766, object='model', owned_by='system'), Model(id='gpt-4o-mini-transcribe', created=1742068596, object='model', owned_by='system'), Model(id='babbage-002', created=1692634615, object='model', owned_by='system'), Model(id='gpt-4o-mini-tts', created=1742403959, object='model', owned_by='system'), Model(id='tts-1-hd-1106', created=1699053533, object='model', owned_by='syste

In [6]:
model_list = json.loads(client.models.list().model_dump_json())["data"]

In total we have 64 models

In [7]:
len(model_list)

64

Along with some information about each model...

In [8]:
model_list[:3]

[{'id': 'gpt-4o-audio-preview-2024-12-17',
  'created': 1734034239,
  'object': 'model',
  'owned_by': 'system'},
 {'id': 'dall-e-3',
  'created': 1698785189,
  'object': 'model',
  'owned_by': 'system'},
 {'id': 'gpt-4o-realtime-preview-2024-12-17',
  'created': 1733945430,
  'object': 'model',
  'owned_by': 'system'}]

But let's just get a list of model names

In [9]:
print("\n".join(sorted([model["id"] for model in model_list])))

babbage-002
chatgpt-4o-latest
dall-e-2
dall-e-3
davinci-002
gpt-3.5-turbo
gpt-3.5-turbo-0125
gpt-3.5-turbo-1106
gpt-3.5-turbo-16k
gpt-3.5-turbo-16k-0613
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
gpt-4
gpt-4-0125-preview
gpt-4-0613
gpt-4-1106-preview
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-4-turbo-preview
gpt-4.5-preview
gpt-4.5-preview-2025-02-27
gpt-4o
gpt-4o-2024-05-13
gpt-4o-2024-08-06
gpt-4o-2024-11-20
gpt-4o-audio-preview
gpt-4o-audio-preview-2024-10-01
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini
gpt-4o-mini-2024-07-18
gpt-4o-mini-audio-preview
gpt-4o-mini-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-mini-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-mini-transcribe
gpt-4o-mini-tts
gpt-4o-realtime-preview
gpt-4o-realtime-preview-2024-10-01
gpt-4o-realtime-preview-2024-12-17
gpt-4o-search-preview
gpt-4o-search-preview-2025-03-11
gpt-4o-transcribe
o1
o1-2024-12-17
o1-mini
o1-mini-2024-09-12
o1-preview
o1-p

## Basic Prompt

The recommended model for exploration is `gpt-3.5-turbo`, so we'll stick with it for now. The basic setup is relatively straightforward:

In [10]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {
            "role": "user", 
            "content": "What was Superman's weakness?"
        },
    ]
)

Which produces a response object

In [11]:
type(response)

openai.types.chat.chat_completion.ChatCompletion

Which we can treat as a `named tuple`. The model answer can be found in the "message" dictionary inside the "choices" list

In [12]:
response.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Superman's primary weakness is Kryptonite - a radioactive mineral from his home planet of Krypton that has the ability to severely weaken and potentially kill him. Additionally, he is also vulnerable to magic and red solar radiation.", role='assistant', function_call=None, tool_calls=None, refusal=None, annotations=[]))

In [13]:
response.choices[0].message.content

"Superman's primary weakness is Kryptonite - a radioactive mineral from his home planet of Krypton that has the ability to severely weaken and potentially kill him. Additionally, he is also vulnerable to magic and red solar radiation."

To request multiple answers, we must include the `n` parameter with the number of answers we want

In [14]:
%%time
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user", 
             "content": "What are the different kinds of Kryptonite?"
        },
    ],
    n=3
)

CPU times: user 2.06 ms, sys: 781 µs, total: 2.84 ms
Wall time: 3.54 s


And we can access each of the answers individually int he choices list

In [15]:
for output in response.choices:
    print("==========")
    print(output.message.role.title()) 
    print("==========")
    print(output.message.content)
    print("==========\n")

Assistant
There are several different kinds of Kryptonite in the DC Comics universe, each with different effects on the character Superman. Some of the most well-known types of Kryptonite include:

1. Green Kryptonite: This is the most common form of Kryptonite and is deadly to Superman. Exposure to green Kryptonite weakens Superman, drains his powers, and can potentially kill him if he is exposed for too long.

2. Red Kryptonite: Red Kryptonite has unpredictable and temporary effects on Superman. Each exposure to red Kryptonite produces a different effect, such as altering his personality, abilities, or physical appearance.

3. Blue Kryptonite: Blue Kryptonite affects Bizarro, a flawed clone of Superman, in the same way that green Kryptonite affects Superman. It has no effect on Superman himself.

4. Gold Kryptonite: Gold Kryptonite permanently removes Superman's powers, rendering him an ordinary human being.

5. White Kryptonite: White Kryptonite is deadly to plant life, causing plan

In [16]:
response.usage

CompletionUsage(completion_tokens=831, prompt_tokens=17, total_tokens=848, prompt_tokens_details={'cached_tokens': 0, 'audio_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0})

In [17]:
print(response.choices[0].message.content)

There are several different kinds of Kryptonite in the DC Comics universe, each with different effects on the character Superman. Some of the most well-known types of Kryptonite include:

1. Green Kryptonite: This is the most common form of Kryptonite and is deadly to Superman. Exposure to green Kryptonite weakens Superman, drains his powers, and can potentially kill him if he is exposed for too long.

2. Red Kryptonite: Red Kryptonite has unpredictable and temporary effects on Superman. Each exposure to red Kryptonite produces a different effect, such as altering his personality, abilities, or physical appearance.

3. Blue Kryptonite: Blue Kryptonite affects Bizarro, a flawed clone of Superman, in the same way that green Kryptonite affects Superman. It has no effect on Superman himself.

4. Gold Kryptonite: Gold Kryptonite permanently removes Superman's powers, rendering him an ordinary human being.

5. White Kryptonite: White Kryptonite is deadly to plant life, causing plants to with

# HuggingFace Basic Usage

HuggingFace relies on pipelines that (mostly) leverage locally run models. All we have to do is specify which task we are interested in and the model we want to use

## Unmasking

Let's look at a simple case of using the base uncase BERT model to fill in masked data. We start by instantiating the pipeline (which will download the model the first time you run it)

In [18]:
unmasker = pipeline('fill-mask', model='bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
output=unmasker("Artificial Intelligence [MASK] take over the world.")
output

[{'score': 0.31824299693107605,
  'token': 2064,
  'token_str': 'can',
  'sequence': 'artificial intelligence can take over the world.'},
 {'score': 0.18299730122089386,
  'token': 2097,
  'token_str': 'will',
  'sequence': 'artificial intelligence will take over the world.'},
 {'score': 0.0560012087225914,
  'token': 2000,
  'token_str': 'to',
  'sequence': 'artificial intelligence to take over the world.'},
 {'score': 0.045194774866104126,
  'token': 2015,
  'token_str': '##s',
  'sequence': 'artificial intelligences take over the world.'},
 {'score': 0.045152731239795685,
  'token': 2052,
  'token_str': 'would',
  'sequence': 'artificial intelligence would take over the world.'}]

After which we can just call the pipeline directly

### Model Bias

As these models are trained on text written by a large number of people, they are also reflective of common biases that are present in society. Depending on our application we may or may not need to take this into account.

In [20]:
unmasker("The man worked as a [MASK].")

[{'score': 0.09747567027807236,
  'token': 10533,
  'token_str': 'carpenter',
  'sequence': 'the man worked as a carpenter.'},
 {'score': 0.05238327011466026,
  'token': 15610,
  'token_str': 'waiter',
  'sequence': 'the man worked as a waiter.'},
 {'score': 0.04962737113237381,
  'token': 13362,
  'token_str': 'barber',
  'sequence': 'the man worked as a barber.'},
 {'score': 0.03788601979613304,
  'token': 15893,
  'token_str': 'mechanic',
  'sequence': 'the man worked as a mechanic.'},
 {'score': 0.037680596113204956,
  'token': 18968,
  'token_str': 'salesman',
  'sequence': 'the man worked as a salesman.'}]

In [21]:
unmasker("The woman worked as a [MASK].")

[{'score': 0.21981653571128845,
  'token': 6821,
  'token_str': 'nurse',
  'sequence': 'the woman worked as a nurse.'},
 {'score': 0.1597415953874588,
  'token': 13877,
  'token_str': 'waitress',
  'sequence': 'the woman worked as a waitress.'},
 {'score': 0.11547262966632843,
  'token': 10850,
  'token_str': 'maid',
  'sequence': 'the woman worked as a maid.'},
 {'score': 0.03796852380037308,
  'token': 19215,
  'token_str': 'prostitute',
  'sequence': 'the woman worked as a prostitute.'},
 {'score': 0.030423782765865326,
  'token': 5660,
  'token_str': 'cook',
  'sequence': 'the woman worked as a cook.'}]

# LangChain

We instantiate the LangChain interface for OpenAI

In [22]:
model = ChatOpenAI(model="gpt-4o")

In [23]:
messages = [
    SystemMessage(content="What was Superman's weakness?"),
]

output = model.invoke(messages)
output

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


AIMessage(content="Superman's primary weakness is Kryptonite, a mineral from his home planet of Krypton. When exposed to it, Superman's powers are significantly weakened, and prolonged exposure can be lethal. Kryptonite comes in various forms, with the most common being green Kryptonite. Other forms, like red, blue, gold, and others, have different effects on Superman, ranging from altering his personality to removing his powers temporarily.\n\nIn addition to Kryptonite, Superman also has vulnerabilities to magic and red solar radiation, which can weaken him or negate his powers.", response_metadata={'token_usage': {'completion_tokens': 110, 'prompt_tokens': 13, 'total_tokens': 123, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_7e8d90e604', 'finish_reason': 'stop', 'logprobs': No

In [24]:
output.response_metadata["token_usage"]

{'completion_tokens': 110,
 'prompt_tokens': 13,
 'total_tokens': 123,
 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
 'completion_tokens_details': {'reasoning_tokens': 0,
  'audio_tokens': 0,
  'accepted_prediction_tokens': 0,
  'rejected_prediction_tokens': 0}}

In [25]:
parser = StrOutputParser()

In [26]:
result = model.invoke(messages)

In [27]:
parser.invoke(result)

"Superman's primary weakness is kryptonite, a radioactive mineral from his home planet of Krypton. Exposure to kryptonite weakens Superman, stripping him of his powers and making him vulnerable. There are different types of kryptonite, each with varying effects on Superman; the most common is green kryptonite, which is harmful and can potentially be lethal with prolonged exposure. Other forms, like red kryptonite, have unpredictable effects, while gold kryptonite can permanently remove his powers."

Let us create our first chain. Stages of the chain are conencted with the pipe '|' character

In [28]:
chain = model | parser

Now whenver we call __invoke()__ on the chain, it automatically runs all the steps

In [29]:
chain.invoke(messages)

"Superman's primary weakness is kryptonite, a radioactive substance from his home planet of Krypton. Exposure to kryptonite weakens Superman, drains his powers, and can potentially be lethal if he is exposed for an extended period. There are different types of kryptonite, each with varying effects, but the most common and well-known is green kryptonite. Additionally, Superman is also vulnerable to magic and can be overpowered by beings with comparable or greater strength."

# Applications

## Text to Code

In [30]:
messages = [
        {
            "role": "system", 
            "content": """You are a grumpy but expert Python software engineer 
            thats interviewing for a job. Please be as concise with your answers as possible."""
        },
        {
            "role": "user", 
            "content": """Create a recursive Python function to compute 
            Fibonacci numbers. Don't provide any explanation, just the code"""
        },
  ]

In [31]:
response = client.chat.completions.create(
    model="gpt-4",
    messages=messages,
    temperature=0,
    max_tokens=1024
)

Which produces the expected result

In [32]:
print(response.choices[0].message.content)

def fibonacci(n):
    if n <= 1:
       return n
    else:
       return(fibonacci(n-1) + fibonacci(n-2))


and works as expected

In [33]:
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return(fibonacci(n-1) + fibonacci(n-2))

In [34]:
fibonacci(10)

55

Let us define a utility function to make sequential queries easier

In [35]:
def chat(messages, prompt):
    messages.append({"role":"user", "content":prompt})
    
    response = client.chat.completions.create(
        model="gpt-4",
        messages=messages,
        temperature=0,
        max_tokens=1024
    )
    
    messages.append(response.choices[0].message)
    
    return messages[-1].content

## Adding comments

In [36]:
print(chat(messages, "Can you add comments to this function?"))

def fibonacci(n):
    # Base case: if n is 0 or 1, return n
    if n <= 1:
       return n
    # Recursive case: return the sum of the previous two Fibonacci numbers
    else:
       return(fibonacci(n-1) + fibonacci(n-2))


In [37]:
print(chat(messages, "What is the purpose of recursion in this piece of code?"))

Recursion in this code is used to break down the problem of calculating a Fibonacci number into smaller, simpler problems. It allows the function to call itself with different arguments, reducing the Fibonacci calculation for a number 'n' to the sum of Fibonacci calculations for 'n-1' and 'n-2', until it reaches the base case where n is 0 or 1.


In [38]:
messages

[{'role': 'system',
  'content': 'You are a grumpy but expert Python software engineer \n            thats interviewing for a job. Please be as concise with your answers as possible.'},
 {'role': 'user',
  'content': "Create a recursive Python function to compute \n            Fibonacci numbers. Don't provide any explanation, just the code"},
 {'role': 'user', 'content': 'Can you add comments to this function?'},
 ChatCompletionMessage(content='def fibonacci(n):\n    # Base case: if n is 0 or 1, return n\n    if n <= 1:\n       return n\n    # Recursive case: return the sum of the previous two Fibonacci numbers\n    else:\n       return(fibonacci(n-1) + fibonacci(n-2))', role='assistant', function_call=None, tool_calls=None, refusal=None, annotations=[]),
 {'role': 'user',
  'content': 'What is the purpose of recursion in this piece of code?'},
 ChatCompletionMessage(content="Recursion in this code is used to break down the problem of calculating a Fibonacci number into smaller, simple

## Explaining Code

Let's use a relatively small python script

In [39]:
code_text = "".join(open("data/EpiModel.py").readlines())

In [40]:
print(code_text)

### −∗− mode : python ; −∗−
# @file EpiModel.py
# @author Bruno Goncalves
######################################################

import networkx as nx
import numpy as np
from numpy import linalg
from numpy import random
import scipy.integrate
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
tqdm.pandas()

class EpiModel(object):
    """Simple Epidemic Model Implementation
    
        Provides a way to implement and numerically integrate 
    """
    def __init__(self, compartments=None):
        self.transitions = nx.MultiDiGraph()
        self.seasonality = None
        
        if compartments is not None:
            self.transitions.add_nodes_from([comp for comp in compartments])
    
    def add_interaction(self, source, target, agent, rate):        
        self.transitions.add_edge(source, target, agent=agent, rate=rate)        
        
    def add_spontaneous(self, source, target, rate):
        self.transitions.add_edge(source, target, rate=rate)



In [41]:
%%time
print(chat(messages, "Please explain what this piece of code does: ```%s```" % code_text))

This code defines a class `EpiModel` for simulating and analyzing epidemic models. The class allows for the creation of a model with multiple compartments (like susceptible, infected, recovered), and transitions between these compartments. 

The class methods allow for adding interactions between compartments, spontaneous transitions, and vaccinations. The model can be simulated stochastically or integrated numerically. 

The `plot` method allows for visualizing the results, and the `R0` method calculates the basic reproduction number of the epidemic. 

The code at the end, under `if __name__ == '__main__':`, is a test run of the class, creating a SIR (Susceptible, Infected, Recovered) model with vaccination, running the simulation, and plotting the results.
CPU times: user 12.4 ms, sys: 2.92 ms, total: 15.3 ms
Wall time: 6.96 s


In [42]:
%%time
print(chat(messages, "Can you please add a doc string to each function and method? Please include information about each argument of the function"))

Sure, here are the docstrings for each method in the `EpiModel` class:

```python
class EpiModel(object):
    """Simple Epidemic Model Implementation"""
    
    def __init__(self, compartments=None):
        """
        Initialize the epidemic model.
        
        Args:
            compartments (list): List of compartments in the model.
        """
        # ...

    def add_interaction(self, source, target, agent, rate):        
        """
        Add an interaction between compartments.
        
        Args:
            source (str): Source compartment.
            target (str): Target compartment.
            agent (str): Agent compartment.
            rate (float): Transition rate.
        """
        # ...

    def add_spontaneous(self, source, target, rate):
        """
        Add a spontaneous transition.
        
        Args:
            source (str): Source compartment.
            target (str): Target compartment.
            rate (float): Transition rate.
        """

## Interacting with a database

Let us open a small test database. This file was downloaded from https://github.com/chineseballer06/Statistical-Analysis-of-Northwind-Database/blob/master/Northwind_small.sqlite

In [43]:
con = sqlite3.connect("data/Northwind_small.sqlite")

In [44]:
messages = [
    {
        "role": "system", 
        "content": """You're a Database Administrator. 
                      Please generate SQL queries to answer the following questions. 
                      No comments are necessary."""
    },
    {
        "role": "user", 
        "content": """
            # Table Employee, columns = [Id, LastName, First Name]
            # Table Shipper, columns = [Id, CompanyName, Phone]
            # Table OrderDetail, columns = [OrderId, ProductId, Quantity]
            # Table EmployeeTerritory, columns = [Id, EmployeeId, TerritoryId]
    """},
]

In [45]:
query_sql = chat(messages, "Generate a table with employee first name, last name and territory id")
print(query_sql)

SELECT Employee.FirstName, Employee.LastName, EmployeeTerritory.TerritoryId
FROM Employee
JOIN EmployeeTerritory ON Employee.Id = EmployeeTerritory.EmployeeId;


In [46]:
pd.read_sql(query_sql, con)

,FirstName,LastName,TerritoryId
0,Nancy,Davolio,06897
1,Nancy,Davolio,19713
2,Andrew,Fuller,01581
3,Andrew,Fuller,01730
4,Andrew,Fuller,01833
5,Andrew,Fuller,02116
6,Andrew,Fuller,02139
7,Andrew,Fuller,02184
8,Andrew,Fuller,40222
9,Janet,Leverling,30346


In [47]:
sql_query = chat(messages, "Compute how many employees work in each territory")
print(sql_query)

SELECT EmployeeTerritory.TerritoryId, COUNT(Employee.Id) as EmployeeCount
FROM Employee
JOIN EmployeeTerritory ON Employee.Id = EmployeeTerritory.EmployeeId
GROUP BY EmployeeTerritory.TerritoryId;


In [48]:
pd.read_sql(sql_query, con)

,TerritoryId,EmployeeCount
0,01581,1
1,01730,1
2,01833,1
3,02116,1
4,02139,1
5,02184,1
6,02903,1
7,03049,1
8,03801,1
9,06897,1


In [49]:
sql_query = chat(messages, "How many shippers do we work with?")
print(sql_query)

SELECT COUNT(*) FROM Shipper;


In [50]:
pd.read_sql(sql_query, con)

,COUNT(*)
0,3


In [51]:
messages

[{'role': 'system',
  'content': "You're a Database Administrator. \n                      Please generate SQL queries to answer the following questions. \n                      No comments are necessary."},
 {'role': 'user',
  'content': '\n            # Table Employee, columns = [Id, LastName, First Name]\n            # Table Shipper, columns = [Id, CompanyName, Phone]\n            # Table OrderDetail, columns = [OrderId, ProductId, Quantity]\n            # Table EmployeeTerritory, columns = [Id, EmployeeId, TerritoryId]\n    '},
 {'role': 'user',
  'content': 'Generate a table with employee first name, last name and territory id'},
 ChatCompletionMessage(content='SELECT Employee.FirstName, Employee.LastName, EmployeeTerritory.TerritoryId\nFROM Employee\nJOIN EmployeeTerritory ON Employee.Id = EmployeeTerritory.EmployeeId;', role='assistant', function_call=None, tool_calls=None, refusal=None, annotations=[]),
 {'role': 'user',
  'content': 'Compute how many employees work in each ter

<center>
     <img src="https://raw.githubusercontent.com/DataForScience/Networks/master/data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>